 #                 [Lab 03] Analise de sentimentos do Twitter

Projeto que tenta analisar como os brasileiros reagiram com a notícia que Neymar nao foi indicado ao título de melhor do mundo



In [41]:
import string
import re,string
import pandas as pd
import numpy as np
import nltk
from collections import Counter
import operator
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

### Lendo dados

Foram coletados quase 60 mil twittes com emojis e classificados como positivos ou negativos, com base nos emojis utilizados. 

Os dados estao no arquivo db.csv e eh utilizado o codigo abaixo para ler esse arquivo.

In [42]:
df = pd.read_csv('db.csv', sep='\t') 


### Coletando uma amostra

Para que possamos trabalhar de forma eficiente (tempo de execucao, memoria), coletamos uma amostra desse data frame

In [43]:
df_0 = df[df['sentiment'] == 0].sample(n = int(0.1*df.shape[0])) #pega 10% dos tweets negativos
df_1 = df[df['sentiment'] == 1].sample(n = int(0.1*df.shape[0])) #pega 10% dos tweets positivos

df = df_0.append(df_1)


### Sera necessario fazer uma limpeza nos tweets

Abaixo estao alguns metodos para limpar os tweets antes de processa-los

In [44]:
def remove_punctuation(text):
    exclude = set(string.punctuation)
    return ''.join(ch for ch in text if ch not in exclude)

def remove_mention(text):
    new_text = ""
    list_of_words = text.split(" ")
    for word in list_of_words:
        if word[0] != "@":
            new_text = new_text + word + " "
    return new_text.rstrip()

def remove_links(text):
    link_regex    = re.compile('((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)', re.DOTALL)
    links         = re.findall(link_regex, text)
    for link in links:
        text = text.replace(link[0], ', ')    
    return text

def remove_pics(text):
    new_text = ""
    list_of_words = text.split(" ")
    for word in list_of_words:
        if len(word) < 32:  ##ver a representacao da imagem melhor para evitar de remover outras palavras
            new_text = new_text + word + " "
    return new_text.rstrip()

def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False
    
def remove_numbers(text):
    new_text = ""
    list_of_words = text.split(" ")
    for word in list_of_words:
        if not(is_number(word)) :
            new_text = new_text + word + " "
    return new_text.rstrip()

def clean_tweet(text):
    return remove_numbers(remove_punctuation(remove_links(remove_pics(remove_mention(text)))))


### Indice invertido

Cria o Indice invertido, como foi visto no Lab01, para que possa 

In [45]:
#cria um mapa de tokens relacionando o indice do tweet e suas palavras

df['tokens'] = df.apply(lambda row: Counter(nltk.word_tokenize(clean_tweet(row['text'].lower()))), axis=1)

# cria o indice invertido a partir dos tokens gerados

inverted_index = {}

for token_list, _id in zip(df.tokens, df.id):
    for token in token_list.keys():
        if token not in inverted_index.keys():
            inverted_index[token] = [_id]
        else:
            inverted_index[token].append(_id)

### TF-IDF

Funcoes para calcular o TF-IDF de cada termo do Tweet

In [46]:
def idf(term):
    N = df.shape[0] # tamanho do corpus
    return np.log(N/len(inverted_index[term.lower().strip()]))

In [47]:
def tf(term):
    return len(inverted_index[term])

In [48]:
def get_tfidfs(tokens_list):
    resp = {}
    for token in tokens_list:
        resp[token] = tf(token)*idf(token)
    return resp

### Topicos

#### Seleciona as palavras mais relevantes daquele tweet

In [49]:
def get_topics(tokens_list, n= 25):
    tfidfs = get_tfidfs(tokens_list)
    
    sorted_d = sorted(tfidfs.items(), key=operator.itemgetter(1),reverse=True)
    
    return [topic[0] for topic in sorted_d[:n]] 

In [50]:
df['topics'] = df.apply(lambda row: get_topics(row['tokens']), axis=1)

In [51]:
df['topics'] = df.apply(lambda row: get_topics(row['tokens']), axis=1)
df_processed = df[['id','topics', 'sentiment']]

In [52]:
df_processed = df_processed.drop('topics', 1).join(df.topics.str.join('|').str.get_dummies(), lsuffix = "_")

### Dividir os dados

Nesse ponto precisamos de uma porcentagem dos dados para treinar o modelo e outra porcentagem para testar o modelo. Escolhi 80% pra treino e 20% pra teste

In [53]:
df_temp = df_processed[[i for i in list(df_processed.columns) if i not in ['id', 'sentiment']]]

X = df_temp.as_matrix()
y = df_processed[['sentiment']].as_matrix()

In [54]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=34)

## Classificador

Aqui criamos o modelo a partir do algoritmo Naive Bayes Multinomial.

In [55]:
mnb = MultinomialNB()
model_mnb = mnb.fit(X_train, y_train.ravel())

In [56]:
from sklearn.naive_bayes import BernoulliNB
bnb = BernoulliNB()
model_bnb = bnb.fit(X_train, y_train.ravel())

## Teste do modelo

Com o modelo ja treinado agore precisamos testa-lo.

In [57]:
#definimos 
y_pred = model_mnb.predict(X_test)
y_true = y_test

In [58]:
y_pred

array([0, 1, 0, ..., 0, 0, 1])

Usaremos inicialmente a acurácia. Que mostra a porcentagem de acertos/numero total

In [59]:

accuracy_score(y_true, y_pred)

0.7250430292598967

Usaremos também matriz de confusão para saber mais detalhes dos acertos e erros

In [60]:
confusion_matrix(y_true,y_pred)

array([[930, 254],
       [385, 755]])

## Melhoramento o modelo

Antes de chegar nesse resultado do modelo, foi necessário muitos testes e mudanças. 

Minha primeira implementação utilizava o algoritmo Gaussian Naive Bayes para classificação pois foi o primeiro que encontrei que utilizava naive bayes. Também utilizei apenas 5% da base de dados (cerca de 3 mil tweets) e selecionava apenas 3 palavras como tópicos de cada tweet. Foi uma escolha que visava a agilidade de processamento e testes durante a programação. Com essas configurações, a acurácia era pouco mais de 50%. E vendo mais detalhes, utilizando a matriz de confusão, percebi que ele classificava quase todos os tweets como positivo. Por isso acertava todos os positivos e errava quase todos os negativos. 

Imaginei inicialmente que poderia ser o número de tópicos, então resolvi aumentar de 3 para 5. De fato, houve uma melhora na acurácia porém continuava abaixo 55%. Aumentei o número de tópicos para 10, depois 20 e então consegui chegar aos 55%. O problema que por mais que eu aumentasse o número de tópicos, nesse ponto o valor não melhorava. Resolvi então aumentar o número da amostra. Agora coletava 2%, porém nesse ponto comecei a ter problemas de estouro de memória. então vi que não adiantava forçar mais, agora a mudança tinha que ser no modelo ou na forma de tratar os tweets. 

Então fiz diversas limpezas nos tweets, removendo assim mentions, links e imagens. Tudo isso proporcionou uma pequena melhoria, mas nada muito mais do que 66% de acurácia. 

Estava muito insatisfeito com meu modelo. Pensei então em utilizar outro algoritmo, uma rede neural, por exemplo. E nessa pesquisa descobri que havia outras variações do naive bayes. E então utilizando o algoritmo Multinominal Naive Bayes foi possível atingir os 75%. 

Provavelmente uma melhor limpeza nos tweets, uma melhor classificação (não utilizando apenas emojis para classificar, adicionar uma nova categoria para os neutros) ou utilizar uma amostra maior, seria possível melhorar essa acurácia. Mas a níveis de estudo acredito que foi encontrado um bom modelo.

## Uso do modelo

Agora que temos um bom modelo, gostaria de testa-lo em um conjunto de 30 tweets, classificados à mão por mim,  sobre a entrevista de bolsonaro no roda viva. Vamos ver como nosso modelo sai em outros contextos. 



Precisamos primeiro pegar esses dados e deixa-los na mesma dimensão que o classificador consiga ler.

In [61]:
new_df = pd.read_csv('bols6.csv', sep='\t')

In [62]:
tweets_test = []

num_columns_to_train = X_train.shape[1]

for tweet_topics in new_df:
    row = np.zeros((df_resp.shape[1],), dtype=int)
    
    #fill the row
    for topic in tweet_topics:
        if topic in (df_processed.columns.values):
            idx = df_processed.columns.get_loc(topic)
            row[idx] = 1
    
    tweets_test.append(row)

NameError: name 'df_resp' is not defined

Agora jogar esse novo data frame no classificador

In [ ]:
result = model.predict(tweets_test)
y_real = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]

Agora vamos ver os resultados

In [ ]:
accuracy_score(y_real, result)

In [ ]:
confusion_matrix(y_true,y_pred)

## Conclusao

Podemos ver, pela acurácia que o modelo 
Também podemos observar, pela matriz de confusão, que o modelo erra